<a href="https://colab.research.google.com/github/copev313/Customer-Segmentation-RFM-Matrix-Principle/blob/main/customer-segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer Segmentation: The RFM Matrix Principle**

## Introduction

In this project we will be demonstrating how the *RFM matrix principle* can be used to investigate customer segmentation, figuring out who our best customers are.

**RFM** -- *Recency, Frequency, Monetary*

This technique uses past customer behavior on purchases to split them into different groups.

## How RFM Works

We will be taking a look at the following three target categories in our analysis:

- **Recency**: how many days since the last purchase?

- **Frequency**: how many total purchases does the customer have?

- **Monetary Value**: what is the total amount of money the customer has spent?

---

## Load & Explore Our Data

In [4]:
import pandas as pd
import warnings


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

retail_data = pd.read_csv('online_retail.csv')

print(retail_data.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

    InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/10 8:26       2.55     17850.0  United Kingdom  
1  12/1/10 8:26       3.39     17850.0  United Kingdom  
2  12/1/10 8:26       2.75     17850.0  United Kingdom  
3  12/1/10 8:26       3.39     17850.0  United Kingdom  
4  12/1/10 8:26       3.39     17850.0  United Kingdom  


In [5]:
# What are the different countries represented in our data?
print(retail_data['Country'].unique())

# How many different countries?
print('\nThere are ', retail_data['Country'].nunique(), ' different countries.')

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Austria' 'Israel' 'Finland' 'Bahrain' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']

There are  38  different countries.


---
Let's find the number of customers per country.

In [6]:
customer_country = retail_data[['Country', 'CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count')\
                .reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


Since the data shows more than 90% of our customer are from the UK only, we will limit out data to United Kingdom only. 

**Some research shows that customer segmentation can vary with geography.*

In [7]:
# Refine our data down to just UK customers:
uk_data = retail_data.loc[retail_data['Country'] == 'United Kingdom']


---

## Data Cleaning

In [8]:
# Check for null values:
uk_data.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

We're missing values from the CustomerID and Description column.

Since we won't be focusing on the customer's ID we can simple remove the missing values from that column.

In [9]:
uk_data = uk_data[pd.notnull(uk_data['CustomerID'])]

In [10]:
# Remove negative values in the Quantity column:
uk_data = uk_data[(uk_data['Quantity'] > 0)]

In [11]:
# Calculate a new column for total price:
uk_data['TotalPrice'] = uk_data['Quantity'] * uk_data['UnitPrice']

Now we need to find the first and last order in our data so that we can calculate ***recency***.

In [12]:
# First date:
print('First Order: ', uk_data['InvoiceDate'].min())

# Last date:
print('Last Order: ', uk_data['InvoiceDate'].max())


First Order:  1/10/11 10:32
Last Order:  9/9/11 9:52


In [13]:
import datetime as dt

# 12/9/2011 was most recent, so we will use the next day for our calculations:
now = dt.datetime(2011, 12, 10)

# Make column datetime dtypes:
uk_data['InvoiceDate'] = pd.to_datetime(uk_data['InvoiceDate'])

---

## RFM Customer Segmentation

Let's start by first creating a RFM table.

In [14]:
rfm_table = uk_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (now - x.max()).days, 
    'InvoiceNo': lambda x: len(x),
    'TotalPrice': lambda x: x.sum()
})

rfm_table['InvoiceDate'] = rfm_table['InvoiceDate'].astype(int)

rfm_table.rename(
    columns = {
        'InvoiceDate': 'recency',
        'InvoiceNo': 'frequency',
        'TotalPrice': 'monetary_value',
    },

    inplace = True
)


# Take a look:
print(rfm_table.head(10))


            recency  frequency  monetary_value
CustomerID                                    
12346.0         325          1        77183.60
12747.0           2        103         4196.01
12748.0           0       4596        33719.73
12749.0           3        199         4090.88
12820.0           3         59          942.34
12821.0         214          6           92.72
12822.0          70         46          948.88
12823.0          74          5         1759.50
12824.0          59         25          397.12
12826.0           2         91         1474.72


---

Now we're going to split our metrics into segments by using quantiles. After, we'll create a customer segment RFM table to convey our best customers.

In [15]:
# Split into quantiles;
quantiles = rfm_table.quantile(q=[0.25, 0.5, 0.75])
quantiles = quantiles.to_dict()

The lowest recency (r), highest frequency (f) and monetary amounts (m) will result in our best customers.

In [16]:
# Create functions to calculate our different target scores.

def r_score(x, p, d):
    if (x <= d[p][0.25]):
        return 1
    elif (x <= d[p][0.50]):
        return 2
    elif (x <= d[p][0.75]): 
        return 3
    else:
        return 4
    
def fm_score(x, p, d):
    if (x <= d[p][0.25]):
        return 4
    elif (x <= d[p][0.50]):
        return 3
    elif (x <= d[p][0.75]): 
        return 2
    else:
        return 1


In [18]:
# Create a table based off our previous:
segmented_rfm  = rfm_table

# Apply our functions to the appropriate columns:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(
    r_score, args=('recency', quantiles, )
)

segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(
    fm_score, args=('frequency', quantiles, )
)

segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(
    fm_score, args=('monetary_value', quantiles, )
)

print(segmented_rfm[['r_quartile', 'f_quartile', 'm_quartile']].head())

            r_quartile  f_quartile  m_quartile
CustomerID                                    
12346.0              4           4           1
12747.0              1           1           1
12748.0              1           1           1
12749.0              1           1           1
12820.0              1           2           2


---

Now, to visualize our findings better, we will find an RFM score for each customer. This will help us see how the segments have split our customer base.

In [19]:
# Create RFM score:
segmented_rfm['RFMScore'] = segmented_rfm['r_quartile'].map(str)\
                            + segmented_rfm['f_quartile'].map(str)\
                            + segmented_rfm['m_quartile'].map(str)

print(segmented_rfm.head(10))

            recency  frequency  monetary_value  r_quartile  f_quartile  \
CustomerID                                                               
12346.0         325          1        77183.60           4           4   
12747.0           2        103         4196.01           1           1   
12748.0           0       4596        33719.73           1           1   
12749.0           3        199         4090.88           1           1   
12820.0           3         59          942.34           1           2   
12821.0         214          6           92.72           4           4   
12822.0          70         46          948.88           3           2   
12823.0          74          5         1759.50           3           4   
12824.0          59         25          397.12           3           3   
12826.0           2         91         1474.72           1           2   

            m_quartile RFMScore  
CustomerID                       
12346.0              1      441  
12747.0  

---


In [20]:
# Display our top 10 customers:
segmented_rfm[segmented_rfm['RFMScore'] == '111'].sort_values(
    'monetary_value', ascending=False

).head(10)


,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111



---
*Special thanks to AMAN KHARWAL for the project idea and guide.*